In [1]:
using JuMP, Gurobi, DataFrames

In [2]:
df = readtable("housing.csv",header=false)
X = Matrix(df[1:end - 1])
y = df[end];

Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] #readtable#232(::Bool, ::Char, ::Array{Char,1}, ::Char, ::Array{String,1}, ::Array{String,1}, ::Array{String,1}, ::Bool, ::Int64, ::Array{Symbol,1}, ::Array{Any,1}, ::Bool, ::Char, ::Bool, ::Int64, ::Array{Int64,1}, ::Bool, ::Symbol, ::Bool, ::Bool, ::DataFrames.#readtable, ::String) at C:\Users\utilisateur\.julia\v0.6\DataFrames\src\deprecated.jl:1045
 [3] (::DataFrames.#kw##readtable)(::Array{Any,1}, ::DataFrames.#readtable, ::String) at .\<missing>:0
 [4] include_string(::String, ::String) at .\loading.jl:522
 [5] include_string(::Module, ::String, ::String) at C:\Users\utilisateur\.julia\v0.6\Compat\src\Compat.jl:88
 [6] execute_request(::ZMQ.Socket, ::IJulia.Msg) at C:\Users\utilisateur\.julia\v0.6\IJulia\src\execute_request.jl:180
 [7] (::Compat.#inner#14{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at C:\Users\utilisateur\.julia\v0.6\Compat\src\Compat.jl:332
 [8] eventloop(::ZMQ.Sock

In [3]:
# Split into training, test sets (75%/25%)
n = length(y)
test_start = round(Int, 0.75 * n)
train_X = X[1:test_start - 1, :]
train_y = y[1:test_start - 1]
test_X = X[test_start:end, :]
test_y = y[test_start:end];

In [4]:
#See the size of training set and test set
println("Size of training set:",size(train_X),size(train_y))
println("Size of test set:",size(test_X),size(test_y))

Size of training set:(379, 13)(379,)
Size of test set:(127, 13)(127,)


In [5]:
function p2linear(X, y)
    # OutputFlag=0 to hide output from solver
    m = Model(solver=GurobiSolver(OutputFlag=0))
    
    p = size(X, 2) #nb of columns

    # Variables
    @variable(m, t)
    @variable(m, β[1:p])
    
    # Constraints
    @constraint(m, norm(y - X * β) <= t)

    # Objective
    @objective(m, Min, t)

    solve(m)

    return getvalue(β)
end
#β
#how to write beta:\beta+tab

p2linear (generic function with 1 method)

In [6]:
β_linear = p2linear(train_X, train_y)

Academic license - for non-commercial use only


13-element Array{Float64,1}:
 -0.182837  
  0.0463545 
  0.0566232 
  0.814289  
 -5.54437   
  6.49243   
 -0.00690827
 -1.03113   
  0.541436  
 -0.0123992 
 -0.524355  
  0.0130461 
 -0.394916  

In [7]:
# Regular
score_regular = norm(test_y - test_X * β_linear)
println("Linear regression score: ",score_regular)
# baseline
train_y_mean = mean(train_y)
score_baseline = norm(test_y - train_y_mean)
println("Baseline score: ",score_baseline)

Linear regression score: 91.31777137966286
Baseline score: 129.2265350398843
